# Binary Classification

In [14]:
import sys
sys.path = ["/Users/kenichi.matsui/Documents/996_github/LightGBM/python-package/"] + sys.path

In [15]:
import lightgbm as lgb
import numpy as np
import numpy.random as rd
import pandas as pd
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold

from IPython.display import display

## Load data

In [16]:
rd.seed(123)
print('Loading data...')
# load or create your dataset
df_train = pd.read_csv('../../binary_classification/binary.train', header=None, sep='\t')
df_test = pd.read_csv('../../binary_classification/binary.test', header=None, sep='\t')

print("df_train.shape:{}, df_test.shape:{}".format(df_train.shape, df_test.shape))
display(df_train.head())

y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)


Loading data...
df_train.shape:(7000, 29), df_test.shape:(500, 29)


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1,0.869,-0.635,0.226,0.327,-0.690,0.754,-0.249,-1.092,0.000,...,-0.010,-0.046,3.102,1.354,0.980,0.978,0.920,0.722,0.989,0.877
1,1,0.908,0.329,0.359,1.498,-0.313,1.096,-0.558,-1.588,2.173,...,-1.139,-0.001,0.000,0.302,0.833,0.986,0.978,0.780,0.992,0.798
2,1,0.799,1.471,-1.636,0.454,0.426,1.105,1.282,1.382,0.000,...,1.129,0.900,0.000,0.910,1.108,0.986,0.951,0.803,0.866,0.780
3,0,1.344,-0.877,0.936,1.992,0.882,1.786,-1.647,-0.942,0.000,...,-0.678,-1.360,0.000,0.947,1.029,0.999,0.728,0.869,1.027,0.958
4,1,1.105,0.321,1.522,0.883,-1.205,0.681,-1.070,-0.922,0.000,...,-0.374,0.113,0.000,0.756,1.361,0.987,0.838,1.133,0.872,0.808


In [5]:


callbacks

[<function lightgbm.callback.early_stopping.<locals>._callback(env)>,
 <function lightgbm.callback.record_evaluation.<locals>._callback(env)>]

In [17]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': ['binary_logloss','auc', ],
    #'metric': ['auc', 'binary_logloss',],
    'num_leaves': 8,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

res_dict = {}
callbacks = [
    lgb.early_stopping(stopping_rounds=100, first_metric_only=True, verbose=True),
    #lgb.record_evaluation(res_dict)
]

# Training settings
FOLD_NUM = 5
fold_seed = 71
folds = StratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=fold_seed)

# Fitting
ret = lgb.cv(params=params,
               train_set=lgb_train,
               folds=folds,
               num_boost_round=5000,
               verbose_eval = 10,
               callbacks=callbacks, 
               eval_train_metric=True,
               # return_boosters=True
               )

#print(f"best_iteration : {model.best_iteration}")
df_ret = pd.DataFrame(ret)
df_ret.tail()

Training until validation scores don't improve for 100 rounds.
[10]	cv_agg's train auc: 0.771145 + 0.00105442	cv_agg's train binary_logloss: 0.612213 + 0.00205895	cv_agg's valid auc: 0.745838 + 0.0139622	cv_agg's valid binary_logloss: 0.621781 + 0.00367387
[20]	cv_agg's train auc: 0.788937 + 0.0018676	cv_agg's train binary_logloss: 0.578536 + 0.00192071	cv_agg's valid auc: 0.75902 + 0.0152219	cv_agg's valid binary_logloss: 0.595008 + 0.00697823
[30]	cv_agg's train auc: 0.803447 + 0.00163385	cv_agg's train binary_logloss: 0.55849 + 0.00148785	cv_agg's valid auc: 0.767319 + 0.0157379	cv_agg's valid binary_logloss: 0.582061 + 0.00949525
[40]	cv_agg's train auc: 0.814445 + 0.00163745	cv_agg's train binary_logloss: 0.54361 + 0.00138999	cv_agg's valid auc: 0.772337 + 0.0147705	cv_agg's valid binary_logloss: 0.57453 + 0.00967318
[50]	cv_agg's train auc: 0.824993 + 0.00127279	cv_agg's train binary_logloss: 0.532171 + 0.00128254	cv_agg's valid auc: 0.772933 + 0.0160191	cv_agg's valid binary_log

,train auc-mean,train auc-stdv,train binary_logloss-mean,train binary_logloss-stdv,valid auc-mean,valid auc-stdv,valid binary_logloss-mean,valid binary_logloss-stdv
99,0.861688,0.002259,0.489460,0.002139,0.773914,0.012689,0.567869,0.009902
100,0.862317,0.002210,0.488741,0.002174,0.773939,0.012767,0.567898,0.009931
101,0.862821,0.002213,0.488106,0.002191,0.773867,0.012762,0.567922,0.009937
102,0.863465,0.002138,0.487379,0.002094,0.773908,0.012827,0.567912,0.009919
103,0.864025,0.002097,0.486689,0.002089,0.774163,0.012813,0.567659,0.009976


In [18]:
oof = model.get_oof_prediction()

NameError: name 'model' is not defined

In [8]:
print(f"roc_auc_score on oof preds: {roc_auc_score(lgb_train.label, oof)}")

roc_auc_score on oof preds: 0.7738457589985172


In [11]:
test_preds_list = model.predict(X_test, num_iteration=model.best_iteration)
test_preds_avg = np.array(test_preds_list).mean(axis=0)

In [14]:
print(f"roc_auc_score on oof preds: {roc_auc_score(y_test, test_preds_avg)}")

roc_auc_score on oof preds: 0.8323819659442725


# Multi-label classification

In [35]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [51]:
rd.seed(123)

# Loading Iris Dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Split dataset for this demonstration.
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                shuffle=True,
                                                random_state=42)

# one hot representation of y_train
max_class_num = y_train.max()+1
y_train_ohe = np.identity(max_class_num)[y_train]
    
# Create LightGBM dataset for train.
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)

# LightGBM parameter
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'metric' : ["multi_logloss", "multi_error", ],
    'num_class': 3,
    'verbosity' : -1,
}
callbacks = [
    lgb.early_stopping(stopping_rounds=100, first_metric_only=False, verbose=True),
]

# Training settings
FOLD_NUM = 5
fold_seed = 71
folds = StratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=fold_seed)
eval_train_metric=True
# # Fitting
ret, model = lgb.cv(params=params,
               train_set=lgb_train,
               folds=folds,
               num_boost_round=1000,
               verbose_eval = 30,
               callbacks=callbacks, 
               eval_train_metric=eval_train_metric,
               return_boosters=True
               )

print(f"best_iteration : {model.best_iteration}")
df_ret = pd.DataFrame(ret)
df_ret.tail()

[30]	cv_agg's train multi_error: 0.031262 + 0.0130705	cv_agg's train multi_logloss: 0.133683 + 0.0162684	cv_agg's valid multi_error: 0.0716168 + 0.0593762	cv_agg's valid multi_logloss: 0.303502 + 0.130628
[60]	cv_agg's train multi_error: 0.0156809 + 0.0114783	cv_agg's train multi_logloss: 0.056532 + 0.014418	cv_agg's valid multi_error: 0.0807077 + 0.0581416	cv_agg's valid multi_logloss: 0.300564 + 0.183565
Early stopping, best iteration is:
[76]	cv_agg's train multi_error: 0.00893938 + 0.00839559	cv_agg's train multi_logloss: 0.0430204 + 0.01407	cv_agg's valid multi_error: 0.072012 + 0.0454085	cv_agg's valid multi_logloss: 0.31329 + 0.213579
best_iteration : 76


,train multi_error-mean,train multi_error-stdv,train multi_logloss-mean,train multi_logloss-stdv,valid multi_error-mean,valid multi_error-stdv,valid multi_logloss-mean,valid multi_logloss-stdv
71,0.011187,0.010050,0.045616,0.014136,0.072012,0.045409,0.310668,0.207221
72,0.011187,0.010050,0.044963,0.014163,0.072012,0.045409,0.310024,0.208473
73,0.011187,0.010050,0.044321,0.014137,0.072012,0.045409,0.310917,0.209640
74,0.011187,0.010050,0.043631,0.014155,0.072012,0.045409,0.311601,0.211938
75,0.008939,0.008396,0.043020,0.014070,0.072012,0.045409,0.313290,0.213579


In [18]:
oof = model.get_oof_prediction()

In [20]:
print(f"accuracy on oof preds: {accuracy_score(lgb_train.label, np.argmax(oof, axis=1))}")

accuracy on oof preds: 0.9285714285714286


In [29]:
test_preds_list = model.predict(X_test, num_iteration=model.best_iteration)
test_preds_avg = np.array(test_preds_list).mean(axis=0)
test_preds = np.argmax(test_preds_avg, axis=1)

In [30]:
print(f"accuracy on oof preds: {accuracy_score(y_test, test_preds)}")

accuracy on oof preds: 1.0


# Regression

In [45]:
from sklearn.metrics import mean_absolute_error

In [60]:
df_train = pd.read_csv('../../regression/regression.train', header=None, sep='\t')
df_test = pd.read_csv('../../regression/regression.test', header=None, sep='\t')

y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)


In [63]:
lgb_train = lgb.Dataset(X_train, y_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'learning_rate': 0.05,
    'num_leaves': 5,
    'metric': ['l1', 'l2'],
    'verbose': -1
}

callbacks = [
    lgb.early_stopping(stopping_rounds=100, first_metric_only=False, verbose=True),
]

# Training settings
FOLD_NUM = 5
fold_seed = 71
folds = StratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=fold_seed)
eval_train_metric=True
# # Fitting
ret, model = lgb.cv(params=params,
               train_set=lgb_train,
               folds=folds,
               num_boost_round=5000,
               verbose_eval = 200,
               callbacks=callbacks, 
               eval_train_metric=eval_train_metric,
               return_boosters=True
               )
print(f"best_iteration : {model.best_iteration}")

df_ret = pd.DataFrame(ret)
df_ret.tail()

Training until validation scores don't improve for 100 rounds.
[200]	cv_agg's train l2: 0.176162 + 0.000827807	cv_agg's train l1: 0.382783 + 0.00175247	cv_agg's valid l2: 0.195144 + 0.00355304	cv_agg's valid l1: 0.403061 + 0.00357333
[400]	cv_agg's train l2: 0.160079 + 0.000714392	cv_agg's train l1: 0.358822 + 0.00174201	cv_agg's valid l2: 0.194141 + 0.00367513	cv_agg's valid l1: 0.395889 + 0.0035235
Early stopping, best iteration is:
[370]	cv_agg's train l2: 0.162128 + 0.000696699	cv_agg's train l1: 0.361787 + 0.00164497	cv_agg's valid l2: 0.194049 + 0.00372737	cv_agg's valid l1: 0.396464 + 0.00359467
best_iteration : 370


,train l2-mean,train l2-stdv,train l1-mean,train l1-stdv,valid l2-mean,valid l2-stdv,valid l1-mean,valid l1-stdv
365,0.162443,0.000678,0.362268,0.001654,0.194072,0.003669,0.396605,0.003521
366,0.162369,0.000675,0.362159,0.001632,0.194072,0.003699,0.396577,0.003556
367,0.162287,0.000681,0.362051,0.001647,0.194070,0.003699,0.396558,0.003545
368,0.162205,0.000698,0.361908,0.001651,0.194057,0.003718,0.396498,0.003579
369,0.162128,0.000697,0.361787,0.001645,0.194049,0.003727,0.396464,0.003595


In [43]:
oof = model.get_oof_prediction()

In [49]:
print(f"mae: {mean_absolute_error(y_train, oof):.5f}")

mae: 0.39822


In [ ]:
test_preds_list = model.predict(X_test, num_iteration=model.best_iteration)
test_preds_avg = np.array(test_preds_list).mean(axis=0)

In [ ]:
y_train

In [ ]:
evals_result = {}  # to record eval results for plotting
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=[lgb_train, lgb_test],
                feature_name=['f' + str(i + 1) for i in range(X_train.shape[-1])],
                categorical_feature=[21],
                evals_result=evals_result,
                verbose_eval=10)

In [ ]:

y_pred_proba_list = proxy.predict(test, num_iteration=best_iteration)
y_pred_proba_avg = np.array(y_pred_proba_list).mean(axis=0)
y_pred = np.argmax(y_pred_proba_avg, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print('Averaging accuracy:', accuracy)

In [ ]:


print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
